# Iniciamos 19h05

In [1]:
# !pip install chart_studio

In [2]:
# !pip install cufflinks

# Exercício Gráfico Fluxo.
Vocês vão utilizar um dos tipos de gráfico do tipo Fluxo (Flow).  
  
Referência: https://www.python-graph-gallery.com/  
  
Em grupo vocês devem decidir qual base aberta vão utilizar para fazer uma demonstracão de um fluxo de dados.  
  
No final da aula de hoje vocês devem ter decidido qual base utilizar e se possível uma tentatativa de como utilizar um dos gráficos de Fluxo.  
  
  

In [30]:
%config IPCompleter.use_jedi = False

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning:

Config option `use_jedi` not recognized by `IPCompleter`.



In [54]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

Dados retirados de:

https://www.kaggle.com/lava18/google-play-store-apps

In [55]:
df = pd.read_csv('/content/googleplaystore.csv')

In [56]:
df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [57]:
df.describe()

,Rating
count,9367.000000
mean,4.193338
std,0.537431
min,1.000000
25%,4.000000
50%,4.300000
75%,4.500000
max,19.000000


In [58]:
def binby(val):
    if val > 1_000_000:
        cat = '1M+'
    elif val > 1_000:
        cat = '1k+'
    elif val > 100:
        cat = '100+'
    elif val > 10:
        cat = '10+'
    else:
        cat = '0+'
    return cat

def convert_to_number(df, col):
    numbers = []
    for value in df[col]:
        if value.isnumeric():
            numbers.append(int(value))
        elif 'M' in value:
            value = value.replace('M','')
            numbers.append(int(float(value)*1000000))
                    
    return numbers

df['Reviews'] = convert_to_number(df, 'Reviews')

df['Reviews'] = df['Reviews'].apply(binby)
print(df['Reviews'].value_counts())

1k+     5197
0+      1892
100+    1562
10+     1485
1M+      705
Name: Reviews, dtype: int64


In [59]:
to_drop = df[df['Installs']=='Free'].index
df = df.drop(to_drop)

df['Installs'] = df['Installs'].str.replace('+','').str.replace(',','').astype(int)
lista = list(df['Installs'].unique())
lista.sort()

def binby(val):
  if val > 1_000_000:
    cat = '1M+'
  elif val > 1_000:
    cat = '1k+'
  elif val > 100:
    cat = '100+'
  elif val > 10:
    cat = '10+'
  else:
    cat = '0+'
  return cat

df['Installs'] = df['Installs'].apply(binby)

print(df['Installs'].value_counts())

1k+     5297
1M+     2832
100+    1237
10+      924
0+       550
Name: Installs, dtype: int64


In [60]:
filtro1  = df['Size'] != 'Varies with device'
filtro2 = df['Size'] != '1,000+'
filtro = filtro1 & filtro2
df = df[filtro]

numbers = []
for value in df['Size']:
        if 'k' in value:
            value = value.replace('k','')
            numbers.append(int(float(value)*1000))
        elif 'M' in value:
            value = value.replace('M','')
            numbers.append(int(float(value)*1000000))

df['Size'] = numbers

def categorizar(val):
  if val < 1_000_000:
    cat = 'Pequeno'
  elif val < 10_000_000:
    cat = 'Medio'
  else:
    cat = 'Grande'
  return cat

df['Size'] = df['Size'].apply(categorizar)
print(df['Size'].value_counts())

Grande     5231
Medio      3599
Pequeno     315
Name: Size, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [61]:
df['Price'] = df['Price'].str.replace('$', '')
index_to_remove = df[df['Price']=='Everyone'].index
df = df.drop(index_to_remove)
df['Price'] = df['Price'].astype(float)
df['Price'] = pd.cut(df['Price'], 3, labels=['baixo', 'medio', 'alto'])
df['Price'] = df['Price'].astype(str)
print(df['Price'].value_counts())

baixo    9126
alto       17
medio       2
Name: Price, dtype: int64


In [62]:
filtro_colunas  = ['Size', 'Installs', 'Reviews','Price']
df_sankey = df[filtro_colunas]

label = []
source = []
target = []
value = []

for i,col in enumerate(filtro_colunas[:-1]):
  col1 = filtro_colunas[i]
  col2 = filtro_colunas[i+1]

  resumo = df_sankey.groupby(by=[col1, col2]).size().reset_index()
  resumo = resumo.rename(columns={0:'Values'})
  resumo[col1] = col1 + '_' + resumo[col1]
  resumo[col2] = col2 + '_' + resumo[col2]
  
  if i == 0:
    label.extend(list(resumo[col1].unique())) 
  label.extend(list(resumo[col2].unique()))
  # print('loop: ',i)
  # print(label)
  # print(col2)
  for index, row in resumo.iterrows():
    source.append(label.index(row[col1]))
    target.append(label.index(row[col2]))
    value.append(row['Values'])  

In [63]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = label,
      color = "blue"
    ),
    link = dict(
      source = source, # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = target,
      value = value
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()